In [1]:
import os
import sys
import importlib
bindingspath = os.getcwd() + '/../build/python'
sys.path.append(bindingspath)
print(sys.path)

['/home/adria/Documents/Documents/TUM/dlfs/libdl/python', '/home/adria/anaconda3/lib/python37.zip', '/home/adria/anaconda3/lib/python3.7', '/home/adria/anaconda3/lib/python3.7/lib-dynload', '', '/home/adria/anaconda3/lib/python3.7/site-packages', '/home/adria/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/adria/.ipython', '/home/adria/Documents/Documents/TUM/dlfs/libdl/python/../build/python']


In [2]:
import pybindings
pybindings.hello("pypy")

0

In [3]:
import numpy as np
aData = np.array([[1, 2], [3, 4],[5,6]]);
print(aData)
print(aData.shape[0])
someObject = pybindings.MyClass()
someObject.set_matrix(aData)
bData = someObject.view_matrix()
bData

[[1 2]
 [3 4]
 [5 6]]
3


array([[1., 2.],
       [3., 4.],
       [5., 6.]])

In [4]:
# Load Mnist
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
mnist = fetch_openml('mnist_784', version=1, cache=True)
trX, teX, trY, teY = train_test_split(mnist.data / 255.0, mnist.target.astype("int0"), test_size = 0.33)
# Data vectors are expected in columns
trX = trX.transpose();
teX = teX.transpose();

In [5]:
# Do one-hot encoding labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
enc.fit(trY.reshape(-1, 1))
labelsOneHot = enc.transform(trY.reshape(-1, 1)).toarray()

In [6]:
print("Values per Image:", trX.shape[0])
print("Number of images for training:", trX.shape[1])
print("--- ---")
print("Training Labels:", labelsOneHot.shape[0])
print("Training Labels:", labelsOneHot.shape[1])

Values per Image: 784
Number of images for training: 46900
--- ---
Training Labels: 46900
Training Labels: 10


In [9]:
import importlib
importlib.reload(pybindings)

<module 'pybindings' from '/home/adria/Documents/Documents/TUM/dlfs/libdl/python/../build/python/pybindings.cpython-36m-x86_64-linux-gnu.so'>

In [11]:
exampleModel = pybindings.ExampleModel()
exampleModel.setInputs(trX,1,28,28)
exampleModel.setLabels(labelsOneHot,10)


In [12]:
exampleModel.trainz()

---------start forward ---------
firstConvLayer.ForwardPass()  ------
mWeights
6 6
(*mInputPtr)
784 1
mOutput
210 6
firstSigmoidLayer.ForwardPass()  ------
secondConvLayer.ForwardPass()  ------
mWeights
36 7
(*mInputPtr)
210 6
mOutput
64 7
secondSigmoidLayer.ForwardPass()  ------
flattenLayer.ForwardPass()  ------
(*mInputPtr)
64 7
mOutput
1 448
fcLayer.ForwardPass()  ------
- (*mInputPtr)
1 448
mOutput
1 10
lossLayer.ForwardPass()  ------
(*mInputPtr)
1 10
(*mInputPtr)
 6.66453  12.8964  9.25108  9.35194  11.4768  5.29542  1.29862  7.54052   12.351 -23.4597
exp
    784.091      398875     10415.8     11521.1     96453.6     199.421     3.66422     1882.81      231184 6.48005e-11
probs
 0.00104362    0.530899   0.0138634   0.0153345    0.128379 0.000265428 4.87705e-06  0.00250601    0.307704 8.62488e-17
      0       0       0       0 2.05277       0       0       0       0       0
lossLayer.GetLoss() ++++++++++++++++++++++++++++++++++++++++++++++++++
2.05277
---------start backward --

mGradientsWeights
6 6
mGradientsBiases
1 6


In [ ]:
from platform import python_version

print(python_version())